In [13]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-25


In [14]:
SOURCE_PATH = f'..\data\{date_str}_df_finviz_stocks_etfs.parquet'
DES_PATH = f'..\data\{date_str}_df_finviz_n_ratios_stocks_etfs.parquet'
COMMON_TICKERS_PATH = f'..\data\{date_str}_df_common_tickers_stocks_etfs.parquet'

In [15]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1500) 
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell

In [16]:
# Load the parquet file into a DataFrame
df_finviz = pd.read_parquet(SOURCE_PATH)

# Display basic information about the DataFrame
print(f"df_finviz.shape: {df_finviz.shape}")
print("\nFirst few rows:")
print(df_finviz.head(20))

df_finviz.shape: (1560, 110)

First few rows:
        No.                                 Company               Index                  Sector                        Industry Country Exchange  Market Cap, M     P/E  Fwd P/E    PEG    P/S    P/B     P/C   P/FCF    Book/sh    Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %       EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta       ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %              52W

In [17]:
print(f'df_finviz.shape before removing duplicates: {df_finviz.shape}')
# Remove duplicates from df_finviz, keeping the first occurrence
df_finviz = df_finviz[~df_finviz.index.duplicated(keep='first')]

print(f'df_finviz.shape after removing duplicates: {df_finviz.shape}')

df_finviz.shape before removing duplicates: (1560, 110)
df_finviz.shape after removing duplicates: (1559, 110)


In [18]:
# Load the parquet file into a DataFrame
df_perf_ratios= pd.read_parquet(f'..\data\{date_str}_df_perf_ratios_stocks_etfs.parquet')

# Display basic information about the DataFrame
print(f"df_perf_ratios.shape: {df_perf_ratios.shape}")
print("\nFirst few rows:")
df_perf_ratios.head()

df_perf_ratios.shape: (1537, 24)

First few rows:


,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
A,5.153101,19.052913,2.697368,15.793771,65.559711,9.259747,3.204522,5.896718,1.619189,1.202716,1.867554,1.226958,-2.257962,-2.893040,0.672414,-3.871887,-4.635301,0.500543,-1.240867,-1.661360,0.804215,-0.846424,-1.118219,0.859558
AA,3.723058,11.141484,1.992562,10.880600,28.880362,4.638584,1.956660,2.954931,1.380855,1.063227,1.742029,1.200455,-2.404414,-3.160253,0.660904,-1.736624,-2.277000,0.738112,-1.411740,-1.848063,0.780096,-0.480904,-0.665725,0.922313
AAL,30.773939,3802.268850,339.733032,21.440824,15666.484213,1974.791483,0.627020,0.870148,1.105977,0.959973,1.686722,1.221849,-0.564022,-0.880908,0.893880,-2.831226,-3.910800,0.571980,-0.808785,-1.219591,0.848549,-0.487410,-0.718294,0.907541
AAON,36.487069,3802.268850,339.733032,47.280299,15666.484213,1974.791483,2.546433,3.924418,1.466622,4.647126,10.956734,2.263572,1.206088,2.089533,1.223265,-1.314690,-1.632439,0.775873,-0.564618,-0.742126,0.891838,0.141787,0.188521,1.029324
AAPL,17.945283,3802.268850,339.733032,25.731176,15666.484213,1974.791483,4.214191,6.719272,1.915683,2.638663,5.831754,1.629695,0.008446,0.013045,1.001637,-0.862372,-1.246825,0.843909,-0.305406,-0.438132,0.939938,0.708262,1.053964,1.149039


In [19]:
# Find items in df_finviz.index that are not in df_perf_ratios.index
missing_tickers = df_finviz.index[~df_finviz.index.isin(df_perf_ratios.index)]
print("Tickers in df_finviz but not in df_perf_ratios:")
print(missing_tickers)
print(f"\nTotal missing tickers: {len(missing_tickers)}")

Tickers in df_finviz but not in df_perf_ratios:
Index(['FER', 'SW', 'VG', 'CRWV', 'VIK', 'ZG', 'LINE', 'TTAN', 'SAIL', 'LTM', 'TEM', 'SARO', 'SFD', 'DFCF', 'BULL', 'WAY', 'TLX', 'NBIS', 'LB', 'SMBS', 'BTC', 'FSEC'], dtype='object', name='Ticker')

Total missing tickers: 22


In [20]:
# Find common tickers (tickers present in both DataFrames' indices)
common_tickers = df_finviz.index[df_finviz.index.isin(df_perf_ratios.index)].tolist() #Convert pandas index to list

print("\nTickers common to both df_finviz and df_perf_ratios:")
print(common_tickers)
print(f"\nTotal common tickers: {len(common_tickers)}")

# Create the empty DataFrame with the common tickers as the index
df_common_tickers = pd.DataFrame(index=common_tickers)
df_common_tickers.to_parquet(COMMON_TICKERS_PATH, engine='pyarrow', compression='zstd')
print(f'Saved df_common_tickers to {COMMON_TICKERS_PATH}')


Tickers common to both df_finviz and df_perf_ratios:
['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'GOOG', 'META', 'BRK-A', 'BRK-B', 'TSLA', 'AVGO', 'TSM', 'LLY', 'WMT', 'JPM', 'V', 'VOO', 'SPY', 'IVV', 'MA', 'XOM', 'NFLX', 'VTI', 'COST', 'ORCL', 'UNH', 'PG', 'JNJ', 'HD', 'ABBV', 'SAP', 'KO', 'BAC', 'QQQ', 'BABA', 'ASML', 'PM', 'PLTR', 'TMUS', 'CRM', 'TM', 'CVX', 'WFC', 'MCD', 'CSCO', 'ABT', 'IBM', 'AZN', 'NVS', 'LIN', 'GE', 'NVO', 'MRK', 'HSBC', 'SHEL', 'NOW', 'T', 'MS', 'AXP', 'ISRG', 'HDB', 'ACN', 'PEP', 'VZ', 'INTU', 'GS', 'RTX', 'RY', 'QCOM', 'DIS', 'UBER', 'BX', 'TMO', 'BKNG', 'AMD', 'ADBE', 'UL', 'PGR', 'AMGN', 'BSX', 'SPGI', 'SONY', 'SCHW', 'VEA', 'TXN', 'VUG', 'CAT', 'PDD', 'MUFG', 'TJX', 'DHR', 'BLK', 'SYK', 'NEE', 'BA', 'IEFA', 'PFE', 'GILD', 'VTV', 'CMCSA', 'HON', 'SNY', 'C', 'UNP', 'VRTX', 'TTE', 'SHOP', 'BND', 'DE', 'SPOT', 'LOW', 'AGG', 'AMAT', 'BHP', 'ARM', 'ADP', 'PANW', 'IBN', 'COP', 'BUD', 'ETN', 'MELI', 'LMT', 'CB', 'SAN', 'TD', 'MMC', 'MDT', 'CRWD', 'GEV', 'KKR', 'ENB

In [21]:
print(f'df_finviz.shape before removing missing tickers: {df_finviz.shape}')
# Remove rows where the index matches missing_tickers
df_finviz = df_finviz.drop(missing_tickers)
print(f'df_finviz.shape after removing missing tickers: {df_finviz.shape}')


df_finviz.shape before removing missing tickers: (1559, 110)
df_finviz.shape after removing missing tickers: (1537, 110)


In [22]:
print(f'df_perf_ratios.shape before removing duplicate tickers: {df_perf_ratios.shape}')
# Remove duplicates from df_perf_ratios, keeping the first occurrence
df_perf_ratios = df_perf_ratios[~df_perf_ratios.index.duplicated(keep='first')]
print(f'df_perf_ratios.shape after removing duplicate tickers: {df_perf_ratios.shape}')


df_perf_ratios.shape before removing duplicate tickers: (1537, 24)
df_perf_ratios.shape after removing duplicate tickers: (1537, 24)


In [23]:
# Find duplicate columns *before* adding suffixes
duplicate_cols = list(set(df_finviz.columns) & set(df_perf_ratios.columns))  # Intersection

if duplicate_cols:
    print(f"Error: Duplicate columns found: {duplicate_cols}")
    raise ValueError(f"Duplicate columns found: {duplicate_cols}")

# Now, perform the join (WITHOUT suffixes)
try:
    df = df_finviz.join(df_perf_ratios)  # Simplified join
    print("Join successful!")
    display(df.head()) # Print first few lines.
    display(df.info())
except ValueError as e:
    print(f"Join failed: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Join successful!


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS this Y %,EPS next Y %,EPS past 5Y %,EPS next 5Y %,Sales past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROI %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,1,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3143820.0,33.27,26.31,3.75,7.94,47.15,58.46,31.98,4.44,3.58,0.49,1.00,2/10/2025,16.11,6.29,1.61,7.58,9.53,15.41,8.88,9.18,3.95,10.13,-2.14,2.61,395760.0,96150.0,2.23,0.03,15040.0,15010.0,99.78,0.09,-3.52,63.86,1.45,0.75,1.85,113.13,27.57,136.52,63.80,0.92,0.88,1.26,1.45,46.52,31.76,24.30,6.24,-5.53,-6.06,-11.27,25.39,-16.43,1.28,9.26,2.43,4.93,3.83,-4.42,-8.06,-16.29,23.68,-19.54,24.46,168.15 - 260.10,-19.54,328873.80,51.95,May 01/a,12/12/1980,Yes,Yes,164000.0,1.49,-1.04,2.08,61.16,0.62,38132498,238.59,208.37,206.20,209.75,206.20,209.28,0.44,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,17.945283,3802.26885,339.733032,25.731176,15666.484213,1974.791483,4.214191,6.719272,1.915683,2.638663,5.831754,1.629695,0.008446,0.013045,1.001637,-0.862372,-1.246825,0.843909,-0.305406,-0.438132,0.939938,0.708262,1.053964,1.149039
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,2913010.0,31.56,26.37,2.22,11.13,9.62,40.71,41.60,40.71,9.63,0.83,3.16,5/15/2025,25.42,12.42,3.22,11.45,12.99,18.45,14.21,14.40,12.27,10.23,12.38,15.04,261800.0,92750.0,3.54,1.11,7430.0,7320.0,98.52,1.47,-0.04,73.30,1.16,0.70,2.02,51.25,18.47,34.29,23.60,1.35,1.34,0.30,0.34,69.41,44.96,35.43,6.54,0.48,-11.76,-8.34,-3.86,-7.03,0.98,12.57,2.43,3.52,4.26,1.19,-5.66,-6.55,13.65,-16.33,13.65,344.79 - 468.35,-16.33,491688.02,55.96,Apr 30/a,3/13/1986,Yes,Yes,228000.0,1.25,-0.08,1.34,25.35,0.74,18803717,482.98,387.30,387.00,392.16,384.60,391.85,1.17,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,22.669054,3802.26885,339.733032,37.156640,15666.484213,1974.791483,0.721671,1.097929,1.120237,3.028055,7.340809,1.773858,0.275150,0.504866,1.053757,-0.684677,-1.127332,0.878684,-0.233682,-0.351408,0.957508,-0.153917,-0.215174,0.972382
NVDA,3,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,2708640.0,37.76,19.68,1.27,20.76,34.25,62.69,44.51,3.24,1.77,0.04,0.04,3/12/2025,1.16,2.94,0.89,48.06,27.39,91.83,29.64,70.88,77.94,81.23,146.26,114.20,130500.0,72880.0,4.97,3.23,24480.0,23420.0,95.70,4.00,-0.13,66.45,1.80,1.12,0.83,262.96,82.20,119.18,81.60,4.44,3.88,0.13,0.13,74.99,62.42,55.85,9.38,-2.42,-22.16,-22.69,34.68,-17.34,2.11,6.82,3.83,6.46,5.41,-3.67,-11.59,-22.61,28.16,-27.51,41.91,78.22 - 153.13,-27.51,332929.99,52.40,Feb 26/a,1/22/1999,Yes,Yes,36000.0,3.89,0.39,1.40,315.04,0.79,249869413,161.59,106.43,106.85,

<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, AAPL to FELG
Columns: 134 entries, No. to Omega 250d
dtypes: float64(117), int64(2), object(15)
memory usage: 1.6+ MB


None

In [24]:
df.to_parquet(DES_PATH, engine='pyarrow', compression='zstd')

print(f"Merged dataframe saved to {DES_PATH}")

Merged dataframe saved to ..\data\2025-04-25_df_finviz_n_ratios_stocks_etfs.parquet
